## ISSAP Notebook for Initial Square Analysis

Use the file tray at left to upload the ISSAP json file.

Then write the filename in the codeblock below where it says 'input_file:', including the .json extension.

Beside each code block (or series of code blocks) there is a 'play' symbol. You click on those in turn, progressing from the top of this notebook to the end. If you click on where it says 'x cells hidden' rather than the play icon, the cells will expand revealing the code.

The file tray at left (file folder icon) sometimes needs to be refreshed to see the latest files (with the tray open, the folder with a reload arrow icon).

In [9]:
# @title File to process
# @markdown Forms support many types of fields.

input_file = 'ISSAP_project_save - 2023-08-12T161845.315.json'  # @param {type: "string"}


# Initialization

This section contains code that sets up our ability to run the R code in the Brainerd Robinson section. Run this only once per session. It might take a minute or three to execute; please be patient and wait for the cells to run before moving on.

In [ ]:
## this takes *forever* ONLY RUN THIS BLOCK AND THE NEXT ONE ONCE
## ignore warnings
## This is to enable R to be interpreted in the Brainerd Robinson section
%load_ext rpy2.ipython

In [ ]:
%%R
install.packages('statnet')

# Initial data extraction and reshaping

This code block performs the initial data extraction. It also defines several functions we use or might want to use.

Run when starting a new session or when you are about to start a second round of analysis on a second json file.



In [24]:
# Not all functions are currently used.

import json
import os
import re

def preprocess_json_data(json_data):
    # Use regular expression to find and remove subdictionaries containing 'DELETE'
    pattern = r'{\\"arti_id\\":\\"arti_\d+\\",\\"name\\":\\"DELETE.+?}'
    for key in json_data:
        for item in json_data[key]:
            json_str = json.dumps(item)
            json_str = re.sub(pattern, '', json_str)
            item.clear()
            item.update(json.loads(json_str))
    return json_data

def extract_subkeys_to_files(json_data, output_folder):
    pattern = r'^ctxt_(\d+)$'

    for key, value_list in json_data.items():
        match = re.match(pattern, key)
        if match:
            value = value_list[0]
            subkeys_data = {
                "filename": value.get("filename", None),
                "square": value.get("square", None),
                "module": value.get("module", None),
                "oribtal_seg": value.get("oribtal_seg", None),
                "agency": value.get("agency", None),
                "interp": value.get("interp", None),
                "artifacts": value.get("artifacts", None),
                "description": value.get("description", None),
                "categories": value.get("categories", None),
            }

            # Create a file with the key name and write the subkeys data into it
            filename = os.path.join(output_folder, f"{key}.json")
            with open(filename, "w") as file:
                json.dump(subkeys_data, file, indent=2)

if __name__ == "__main__":
    file = input_file

    with open(file, "r") as json_file:
        json_data = json.load(json_file)

    # Preprocess the JSON data to remove subdictionaries containing 'DELETE'
    json_data = preprocess_json_data(json_data)

    output_folder = "output_folder"
    os.makedirs(output_folder, exist_ok=True)

    extract_subkeys_to_files(json_data, output_folder)

### Centroids

import re
import json
import networkx as nx
import matplotlib.pyplot as plt
import csv
import community

def calculate_centroid(polygon_points):
    # Extract the coordinates from the string
    coordinates = re.findall(r'(\d+\.\d+),(\d+\.\d+)', polygon_points)
    num_points = len(coordinates)

    # Initialize variables for centroid coordinates
    centroid_x = 0
    centroid_y = 0

    # Calculate the sum of x and y coordinates
    for point in coordinates:
        x, y = map(float, point)
        centroid_x += x
        centroid_y += y

    # Calculate the average of x and y coordinates
    centroid_x /= num_points
    centroid_y /= num_points

    return centroid_x, centroid_y

# or use shapely for centroids
from shapely.geometry import Polygon

def calculate_centroid_shapely_method(polygon_points):
    # Extract the coordinates from the string
    coordinates = re.findall(r'(\d+\.\d+),(\d+\.\d+)', polygon_points)

    # Create a Shapely polygon object
    polygon = Polygon(coordinates)

   # Calculate the centroid using the Shapely library
    centroid = polygon.centroid

    # Return the centroid coordinates
    return centroid.x, centroid.y

# or use polygon area method
def calculate_centroid_pam(polygon_points):
    # Extract the coordinates from the string
    coordinates = re.findall(r'(\d+\.\d+),(\d+\.\d+)', polygon_points)

    # Convert the coordinates to floats
    vertices = [(float(x), float(y)) for x, y in coordinates]

    # Calculate the signed area and centroid coordinates
    signed_area = 0.0
    centroid_x = 0.0
    centroid_y = 0.0

    for i in range(len(vertices)):
        x_i, y_i = vertices[i]
        x_next, y_next = vertices[(i + 1) % len(vertices)]  # Wrap around to the first vertex

        # Compute the cross product of consecutive vertices
        cross_product = (x_i * y_next) - (x_next * y_i)

        # Accumulate the signed area and centroid coordinates
        signed_area += cross_product
        centroid_x += (x_i + x_next) * cross_product
        centroid_y += (y_i + y_next) * cross_product

    # Divide by 6 times the signed area to get the centroid coordinates
    signed_area *= 0.5
    centroid_x /= (6.0 * signed_area)
    centroid_y /= (6.0 * signed_area)

    # Return the centroid coordinates
    return centroid_x, centroid_y


def create_centroid_network(centroid_list, distance_threshold):
    # Create an empty graph
    G = nx.Graph()

    # Add nodes to the graph
    for i, centroid in enumerate(centroid_list):
        arti_id = centroid['name']
        G.add_node(arti_id, centroid=centroid)

    # Calculate distances and add edges between centroids
    for i in range(len(centroid_list)):
        for j in range(i+1, len(centroid_list)):
            centroid1 = centroid_list[i]
            centroid2 = centroid_list[j]
            distance = ((centroid1['centroid'][0] - centroid2['centroid'][0])**2 + (centroid1['centroid'][1] - centroid2['centroid'][1])**2)**0.5
            if distance <= distance_threshold:
                arti_id1 = centroid1['name']
                arti_id2 = centroid2['name']
                G.add_edge(arti_id1, arti_id2, distance=distance)

    return G

def export_graph_as_csv(graph, filepath):
    with open(filepath, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Source', 'Target', 'Distance'])

        for edge in graph.edges(data=True):
            source = edge[0]
            target = edge[1]
            distance = edge[2]['distance']
            writer.writerow([source, target, distance])

    print(f"Graph exported as CSV file: {filepath}")

import numpy as np
from scipy.spatial import ConvexHull

def calculate_centroid_ConvexHull_method(polygon_points):
    # Extract the coordinates from the string
    coordinates = re.findall(r'(\d+\.\d+),(\d+\.\d+)', polygon_points)

    # Convert the coordinates to a NumPy array
    points = np.array(coordinates, dtype=float)

    # Calculate the convex hull
    hull = ConvexHull(points)

    # Get the vertices of the convex hull
    hull_vertices = points[hull.vertices]

    # Calculate the centroid of the convex hull
    centroid = np.mean(hull_vertices, axis=0)

    # Return the centroid coordinates
    return centroid[0], centroid[1]

import re
import matplotlib.pyplot as plt

def plot_polygons_with_centroids(polygon_data):
    # Remove backslashes from the polygon data
    polygon_data = polygon_data.replace('\\', '')

    # Extract polygon data using regular expressions
    matches = re.findall(r'<polygon points="([^"]+)"', polygon_data)

    # Create a figure and axes
    fig, ax = plt.subplots()

    for polygon_points in matches:
        # Extract the coordinates from the string
        coordinates = re.findall(r'(\d+\.\d+),(\d+\.\d+)', polygon_points)
        x_coords, y_coords = zip(*[(float(x), float(y)) for x, y in coordinates])

        # Calculate the centroid
        centroid_x = sum(x_coords) / len(x_coords)
        centroid_y = sum(y_coords) / len(y_coords)

        # Plot the polygon
        ax.plot(x_coords, y_coords, 'b-')

        # Plot the centroid
        ax.plot(centroid_x, centroid_y, 'ro')

    # Set the plot title and labels
    ax.set_title('Polygons with Centroids')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')

    # Show the plot
    plt.show()

## close any polygons

import re

def identify_unclosed_polygons(polygon_data):
    # Remove backslashes from the polygon data
    polygon_data = polygon_data.replace('\\', '')

    # Extract polygon data using regular expressions
    matches = re.findall(r'<polygon points="([^"]+)"', polygon_data)

    unclosed_polygons = []

    for i, polygon_points in enumerate(matches):
        # Extract the coordinates from the string
        coordinates = re.findall(r'(\d+\.\d+),(\d+\.\d+)', polygon_points)

        # Check if the polygon is closed
        if coordinates[0] != coordinates[-1]:
            unclosed_polygons.append(i)

    return unclosed_polygons

import re

def close_polygons(polygon_data):
    # Remove backslashes from the polygon data
    polygon_data = polygon_data.replace('\\', '')

    # Extract polygon data using regular expressions
    matches = re.findall(r'<polygon points="([^"]+)"', polygon_data)

    closed_polygon_data = polygon_data

    for polygon_points in matches:
        # Extract the coordinates from the string
        coordinates = re.findall(r'(\d+\.\d+),(\d+\.\d+)', polygon_points)

        # Check if the polygon is closed
        if coordinates[0] != coordinates[-1]:
            # Add the missing coordinate to close the polygon
            closed_polygon_points = polygon_points + f' {coordinates[0][0]},{coordinates[0][1]}'
            # Replace the original polygon points with the closed polygon points
            closed_polygon_data = closed_polygon_data.replace(polygon_points, closed_polygon_points)

    return closed_polygon_data

# Function to close polygons in a JSON file and save the result to a new file
def process_json_file(file_path, output_folder):
    with open(file_path, 'r') as file:
        polygon_data = file.read()

    # Close polygons
    closed_polygon_data = close_polygons(polygon_data)

    # Create the subfolder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Save the closed polygon data to a new file in the subfolder
    filename = os.path.basename(file_path)
    new_file_path = os.path.join(output_folder, 'closed_' + filename)
    with open(new_file_path, 'w') as file:
        file.write(closed_polygon_data)

   # print("Polygons closed and saved to:", new_file_path)

# Folder path containing JSON files
folder_path = 'output_folder'
# Subfolder to save the closed polygon data
output_folder = 'closed_polygon_data'

# Iterate over all JSON files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        process_json_file(file_path, output_folder)

## image functions

import pandas as pd
import json

# Function to extract image width and height in pixels
# question is, which bit of metadata is the correct metadata
def extract_image_dimensions(json_data):
    results = []
    for key in json_data:
        if 'ctxt_' in key:
            try:
                # Get the values of 'Image Width' and 'Image Height' keys
                image_width = json_data[key][0]['exifInfo']['Image Width']['value']
                image_height = json_data[key][0]['exifInfo']['Image Height']['value']
                image_width2 = json_data[key][0]['exifInfo']['ImageWidth']['value']
                image_length = json_data[key][0]['exifInfo']['ImageLength']['value']
                PixelXDimension = json_data[key][0]['exifInfo']['PixelXDimension']['value']
                PixelYDimension = json_data[key][0]['exifInfo']['PixelYDimension']['value']

                # Extract ctxt key number
                ctxt_num = key.split("_")[1]  # Assumes ctxt is of format ctxt_x

                results.append((ctxt_num, image_width, image_height, image_width2, image_length, PixelXDimension, PixelYDimension))
            except KeyError:
                results.append((None, None, None))
    return results

output_folder = "analyzed_folder"
if not os.path.exists(output_folder):
        os.makedirs(output_folder)


# Initialize an empty DataFrame
imagedf = pd.DataFrame(columns=['ctxt', 'image_width', 'image_height', 'image_width2', 'image_length', 'PixelXDimension','PixelYDimension'])

# Open the file and read the lines
with open(input_file, 'r') as f:
    # Read the file line by line
    for line in f:
        # Load JSON data from each line
        data = json.loads(line)

        # Extract ctxt number, image width and length
        dimensions = extract_image_dimensions(data)

        # Append to the DataFrame
        for ctxt, width, height, width2, length, pixelxdimension, pixelydimension in dimensions:
            imagedf = pd.concat([imagedf, pd.DataFrame({'ctxt': [ctxt], 'image_width': [width], 'image_height': [height], 'image_width2': [width2], 'image_length': [length], 'PixelXDimension': [pixelxdimension], 'PixelYDimension': [pixelydimension]})], ignore_index=True)
            imagedf.to_csv(f'analyzed_folder/image_pixel_dimension_info.csv', index=False)

## centroid functions for working with network graphs, should we do that
#centroid functions
import os
import json
import numpy as np
import networkx as nx
from scipy.spatial.distance import cdist
from scipy.spatial import distance_matrix
import pandas as pd

def calculate_relative_neighborhood(centroids, threshold):
    # Calculate the pairwise distances between centroids
    distances = cdist(centroids, centroids, metric='minkowski', p=1)

    # Create an empty graph
    G = nx.Graph()

    # Add nodes to the graph
    num_centroids = len(centroids)
    for i in range(num_centroids):
        G.add_node(i, pos=(centroids[i][0], centroids[i][1]))

    # Add edges to the graph based on the relative neighborhood criterion
    for i in range(num_centroids):
        for j in range(i + 1, num_centroids):
            if distances[i, j] <= threshold:
                G.add_edge(i, j)

    return G

## rng per JB http://proceedings.caaconference.org/files/2011/42_Jimenez-Badillo_CAA2011.pdf
def calculate_relative_neighbourhood_jb(centroids, beta=1.0):
    # Initialize an empty graph
    graph = nx.Graph()

    # Compute the distance matrix
    dist_matrix = distance_matrix(centroids, centroids)

    # Check all pairs of points
    for i in range(len(centroids)):
        for j in range(i + 1, len(centroids)):
            # Get the distance between points i and j, adjusted by beta
            d = beta * dist_matrix[i, j]

            # Check if any other point lies within the region RRNG
            inside_rrng = False
            for k in range(len(centroids)):
                if k != i and k != j and max(dist_matrix[i, k], dist_matrix[j, k]) < d:
                    inside_rrng = True
                    break

            # If no point is inside the RRNG, add an edge between i and j
            if not inside_rrng:
                graph.add_edge(i, j)

    return graph

# Function to process a single JSON file and create the network, community DataFrame, and name DataFrame
def process_json_file(file_path):
    with open(file_path, 'r') as file:
        polygon_data = file.read()

    # Your existing code to extract arti_id, name, and polygon_data
# Remove backslashes from the polygon data
    polygon_data = polygon_data.replace('\\', '')

# matches = re.findall(r'<polygon points="([^"]+)"', polygon_data)
    matches = re.findall(r'"arti_id":"([^"]+)".*?"name":"([^"]+)".*?"type":"([^"]+)".*?"subtype":"([^"]+)".*?<polygon points="([^"]+)"', polygon_data)

    # Calculate centroids for each polygon and store arti_id and name in centroid_list
    centroid_list = []
    for match in matches:
        arti_id, name, atype, subtype, polygon_points = match
        centroid = calculate_centroid(polygon_points) ## change this line to use the calculate_centroid_pam or calculate_centroid_ConvexHull_method functions if you prefer
        centroid_list.append({'arti_id': arti_id, 'name': name, 'type': atype, 'subtype': subtype, 'centroid': centroid})


# Calculate the number of pixels wide using the right-most and left-most centroids
    centroids = [centroid['centroid'] for centroid in centroid_list]  # Add this line
    x_coords = [centroid[0] for centroid in centroids]
    #num_pixels_wide = max(x_coords) - min(x_coords)

## trying to add the actual widths
    distance_threshold_percentage = 20  # Set the desired percentage here (e.g., 20%)

# Loop through your contexts (assuming they are numbered sequentially starting at 0)
    for i in range(len(imagedf)):
    # Read the image_width value from the imagedf DataFrame
        num_pixels_wide = imagedf.loc[i, 'image_width']

    # If the image width value is not None and is a number
        if num_pixels_wide is not None and isinstance(num_pixels_wide, (int, float)):
        # Set the distance threshold as a percentage of the number of pixels wide
            distance_threshold = (distance_threshold_percentage / 100) * num_pixels_wide
            #print(f"threshold for ctxt-{i} is {distance_threshold}") #debugging statement
        else:
        # Handle the case where the image width is not available
            print(f"Image width not available for ctxt_{i}")
            num_pixels_wide = max(x_coords) - min(x_coords)
            distance_threshold = (distance_threshold_percentage / 100) * num_pixels_wide


# Create relative neighborhood network based on centroids
    relative_neighborhood_network = calculate_relative_neighborhood(centroids, distance_threshold)
    rng_jb = calculate_relative_neighbourhood_jb(centroids, 1.1) #higher beta, less connected
   # Debugging print statements
#    print("Number of nodes in the relative_neighborhood_network:", len(relative_neighborhood_network.nodes))
#    print("Number of edges in the relative_neighborhood_network:", len(relative_neighborhood_network.edges))

# communities?
    partition = nx.community.greedy_modularity_communities(relative_neighborhood_network, resolution=1)
# Debugging print statements
#    print("Number of communities detected:", len(partition))

# Add community information to node attributes
    for community_idx, community_nodes in enumerate(partition):
        for node in community_nodes:
            arti_id = centroid_list[node]['arti_id']
            name = centroid_list[node]['name']
            relative_neighborhood_network.nodes[node]['community'] = community_idx
            relative_neighborhood_network.nodes[node]['arti_id'] = arti_id
            relative_neighborhood_network.nodes[node]['name'] = name
            relative_neighborhood_network.nodes[node]['type'] = atype
            relative_neighborhood_network.nodes[node]['subtype'] = subtype

# Create a table for the partition and its communities
    data = []
    for community_idx, community_nodes in enumerate(partition):
        community_info = {
            'Community Index': community_idx,
            'Nodes (Centroids)': [node for node in community_nodes],
            'arti_id': [centroid_list[node]['arti_id'] for node in community_nodes],
            'Name': [centroid_list[node]['name'] for node in community_nodes],
            'Type': [centroid_list[node]['type'] for node in community_nodes],
            'Subtype': [centroid_list[node]['subtype'] for node in community_nodes]
        # Add more attributes as needed
        }
        data.append(community_info)


# Convert the list of dictionaries to a pandas DataFrame
    community_df = pd.DataFrame(data)

# Function to combine 'Name', 'Type', and 'Subtype'
    def combine_names(row):
      return ' '.join([f"{name} {typ} {sub}," for name, typ, sub in zip(row['Name'], row['Type'], row['Subtype'])])

# Apply the function to create a new column 'the_things'
    community_df['the_things'] = community_df.apply(combine_names, axis=1)

# Flatten the 'the_things' column for counting
    flattened_names = [name for sublist in community_df['the_things'] for name in sublist.split(',')]

# aug 16 creating output just for subtypes, per Walsh request
    # Function to combine 'Subtype'
    def combine_subtypes(row):
      return ' '.join([f"{sub}," for sub in row['Subtype']])

# Apply the function to create a new column 'the_subtypes'
    community_df['the_subtypes'] = community_df.apply(combine_subtypes, axis=1)

# Flatten the 'the_subtypes' column for counting
    flattened_subtypes = [subtype for sublist in community_df['the_subtypes'] for subtype in sublist.split(',')]

# Calculate the count of each unique element in the flattened list
    subtype_counts = pd.Series(flattened_subtypes).value_counts().reset_index()
    subtype_counts.columns = ['Subtype', 'Count']


# Calculate the count of each unique element in the flattened list
    name_counts = pd.Series(flattened_names).value_counts().reset_index()
    name_counts.columns = ['Name', 'Count']
    return relative_neighborhood_network, community_df, name_counts, subtype_counts, rng_jb,



# Basic Counts

The following code will create a subfolder within the 'analyzed_folder' with artefact counts by name, type, and subtype, including a summary table that can be further investigated.

+ Artefacts by ctxt are in analyzed_folder/artefact_data
+ Summary counts outputted to analyzed_folder/artefact_data/summary_file.csv

The summary csv can be filtered by context, name, type, subtype, or a pivot table created as appropriate.

In [ ]:
import csv
import json
import re
import os

def process_json_file_for_artefacts(file_path, output_file_path):
    with open(file_path, 'r') as file:
        polygon_data = file.read()

    # Remove backslashes from the polygon data
    polygon_data = polygon_data.replace('\\', '')
    matches = re.findall(r'"arti_id":"([^"]+)".*?"name":"([^"]+)".*?"type":"([^"]+)".*?"subtype":"([^"]+)".*?"material":"([^"]+)".*?"colour":"([^"]+)".*?"fixed":"([^"]+)".*?"persistence":"([^"]+)".*?"orientation":"([^"]+)".*?"artiNotes":"([^"]+)".*?<polygon points="([^"]+)"', polygon_data)

    artefact_list = []
    for match in matches:
        arti_id, name, atype, subtype, material, colour, fixed, persistence, orientation, artiNotes, polygon_points = match
        artefact_list.append([arti_id, name, atype, subtype, material, colour, fixed, persistence, orientation, artiNotes, polygon_points])

    # Writing to a csv file
    with open(output_file_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Arti_id", "Name", "Type", "Subtype", "Material", "Colour", "Fixed", "Persistence", "Orientation", "artiNotes","polygon_data"])
        writer.writerows(artefact_list)

def process_all_files_in_subfolder(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for root, dirs, files in os.walk(input_folder):
        for filename in files:
            if filename.endswith('.json'):
                file_path = os.path.join(root, filename)
                output_file_path = os.path.join(output_folder, f"{filename}_artefacts.csv")
                process_json_file_for_artefacts(file_path, output_file_path)

input_folder = "closed_polygon_data"
output_folder = "analyzed_folder/artefact_data"
process_all_files_in_subfolder(input_folder, output_folder)

import csv
import os
from collections import defaultdict

def create_summary_csv(output_folder, summary_file_path):
    summary_data = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

    for root, _, files in os.walk(output_folder):
        for filename in files:
            if filename.endswith('_artefacts.csv'):
                file_path = os.path.join(root, filename)
                with open(file_path, 'r', newline='') as csvfile:
                    reader = csv.DictReader(csvfile)
                    for row in reader:
                        artiId = row ['Arti_id']
                        name = row['Name']
                        atype = row['Type']
                        subtype = row['Subtype']
                        material = row['Material']
                        colour = row['Colour']
                        fixed = row['Fixed']
                        persistence = row['Persistence']
                        orientation = row['Orientation']
                        artiNotes = row['artiNotes']
                        #summary_data[filename]['Arti_id'][artiId] += 1
                        summary_data[filename]['Name'][name] += 1
                        summary_data[filename]['Type'][atype] += 1
                        summary_data[filename]['Subtype'][subtype] += 1
                        summary_data[filename]['Material'][material] += 1
                        summary_data[filename]['Colour'][colour] += 1
                        summary_data[filename]['Fixed'][fixed] += 1
                        summary_data[filename]['Persistence'][persistence] += 1
                        summary_data[filename]['Orientation'][orientation] += 1
                        summary_data[filename]['artiNotes'][artiNotes] += 1

    # Writing the summary data to a CSV file
    with open(summary_file_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['File', 'Category', 'Item', 'Count'])
        for filename, categories in summary_data.items():
            for category, items in categories.items():
                for item, count in items.items():
                    writer.writerow([filename, category, item, count])

summary_file_path = "analyzed_folder/artefact_data/summary_file.csv"
create_summary_csv(output_folder, summary_file_path)
print("Artefacts by ctxt are in analyzed_folder/artefact_data.")
print("summary counts outputted to analyzed_folder/artefact_data/summary_file.csv")
print("The summary csv can be filtered by context, name, type, subtype, material, colour, fixed, persistence, orientation, artiNotes, or a pivot table created as appropriate.")

# Movement

The following section tracks the persistence of artefacts from one context to the next. Some of the reshaping of data that occurs is important for when we push the data to the graph database.

In [26]:
import pandas as pd
import csv
import os
import numpy as np

def calculate_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

def preprocess_files_for_centroid_comparison_step(artefact_path):
    artefacts_dfs = []
    distance_threshold = 50  # to be adjusted per your dataset

    for file in os.listdir(artefact_path):
        if file.endswith('artefacts.csv'):
            filepath = os.path.join(artefact_path, file)
            context = file.split('_')[2].split('.')[0]

            with open(filepath, 'r') as fp:
                csv_reader = csv.reader(fp)
                header = next(csv_reader)
                rows_list = []
                for row in csv_reader:
                    # Same as before...
                    arti_id, name, atype, subtype, material, colour, fixed, persistence, orientation, artiNotes, polygon_points = row
                    polygon_points = polygon_points.replace('"', '')
                    centroid = calculate_centroid(polygon_points)
                    centroid_string = '(' + ', '.join(str(num) for num in centroid) + ')'
                    rows_list.append({'arti_id': arti_id, 'Artefact': name + '-' + atype + '-' + subtype, 'centroid': centroid_string, 'orientation': orientation, 'colour': colour, 'fixed': fixed, 'persistence': persistence, 'artiNotes': artiNotes, 'context': context})

            data = pd.DataFrame(rows_list)

            artifacts_coordinates = {}
            count_artifact = {}
            for _, row in data.iterrows():
                artifact = row['Artefact']
                centroid = tuple(map(float, row['centroid'].strip('()').split(',')))

                is_close_to_another = False
                for artifact_comp, centroid_comp in artifacts_coordinates.items():
                    distance = calculate_distance(centroid, centroid_comp)
                    if distance < distance_threshold:
                        is_close_to_another = True
                        common_base_name = artifact_comp
                        break

                if is_close_to_another:
                    row['Artefact'] = common_base_name + str(count_artifact[common_base_name])
                    count_artifact[common_base_name] += 1
                else:
                    artifacts_coordinates[artifact] = centroid
                    count_artifact[artifact] = 0

            data['Artefact'] = data['Artefact'] + data.groupby('Artefact').cumcount().add(1).astype(str)
            artefacts_dfs.append(data)

    return pd.concat(artefacts_dfs, ignore_index=True)

def replace_orientation_errors(df):
    df['orientation'] = df['orientation'].replace('o', 0)
    return df

def preprocess_files_for_finding_uniques_step(artefact_path):
    artefacts2_dfs = []
    for file in os.listdir(artefact_path):
        if file.endswith('json_artefacts.csv'):
            filepath = os.path.join(artefact_path, file)
            context = file.split('_')[2].split('.')[0] # extracts 'ctxt_01' portion from filename

            with open(filepath, 'r') as fp:
              csv_reader = csv.reader(fp)
              header = next(csv_reader)  # Skip the header row
              rows_list = []
              for row in csv_reader:
                  arti_id, name, atype, subtype, material, colour, fixed, persistence, orientation, artiNotes, polygon_points = row
                  polygon_points = polygon_points.replace('"', '')  # Remove quotes
                  centroid = calculate_centroid(polygon_points)
                  centroid_string = '(' + ', '.join(str(num) for num in centroid) + ')'
                  rows_list.append({'Artefact': name + '-' + atype + '-' + subtype, 'centroid': centroid_string, 'orientation': orientation, 'colour': colour, 'fixed': fixed, 'persistence': persistence, 'artiNotes': artiNotes, 'context': context})

            # convert list of dict to dataframe
            # generate 'rows_list' and convert to DataFrame
        data = pd.DataFrame(rows_list)
        artefacts2_dfs.append(data)

# First, concatenate all data
    all_data = pd.concat(artefacts2_dfs)

# Then, check for unique 'Artefact' values across all data
    counts = all_data['Artefact'].value_counts()
    unique_artefacts = counts[counts == 1].index
    all_data = all_data[all_data['Artefact'].isin(unique_artefacts)]
    return all_data





In [27]:
artefact_path = "analyzed_folder/artefact_data"
final_df = preprocess_files_for_centroid_comparison_step(artefact_path)
final_df = replace_orientation_errors(final_df)
final_df.to_csv(f'{artefact_path}' + '/processed_artefacts.csv', index=False)

# find uniques
uniques_df = preprocess_files_for_finding_uniques_step(artefact_path)
uniques_df = replace_orientation_errors(uniques_df)
uniques_df.to_csv(f'{artefact_path}' + '/uniques.csv', index=False)

In [28]:
## creating output that can be used as input for the notebook for creating the
## graph database
import pandas as pd

def create_artefact_df(final_df):
    final_df['context'] = final_df['context'].astype(int)
    final_df['composite_id'] = 'ctxt_' + final_df['context'].astype(str) + '_'+ final_df['arti_id']
    # Combine 'artifact' descriptions with composite_ids as a tuple
    final_df['artefact_with_id'] = list(zip(final_df['Artefact'], final_df['composite_id']))
    # Group by 'context' and aggregate the descriptions with their composite_ids
    new_df = final_df.groupby('context')['artefact_with_id'].apply(list).reset_index()
    new_df.sort_values('context', ascending=True, inplace=True)
    return new_df

def compare_context_artefacts(comparing_objects_df, context_i, context_j):
    # Get the artefacts for each context
    artefacts_i = dict(comparing_objects_df.loc[comparing_objects_df['context'] == context_i, 'artefact_with_id'].values[0])
    artefacts_j = dict(comparing_objects_df.loc[comparing_objects_df['context'] == context_j, 'artefact_with_id'].values[0])

    # Lists to keep track of artefacts
    only_in_i = []
    only_in_j = []
    in_both = []
    shared_items = []

    # Comparisons
    for artefact, comp_id in artefacts_i.items():
        if artefact not in artefacts_j:
            only_in_i.append((artefact, comp_id))
        else:
            in_both.append((artefact, comp_id))
            shared_items.append((artefact, comp_id, artefacts_j[artefact]))

    for artefact, comp_id in artefacts_j.items():
        if artefact not in artefacts_i:
            only_in_j.append((artefact, comp_id))

    return only_in_i, in_both, only_in_j, shared_items

def iterate_context_pairs(comparing_objects_df):
    shared_results = []
    results = []

    context_values = comparing_objects_df["context"].unique()
    context_values.sort()

    for i in range(len(context_values) - 1):
        context_i = context_values[i]
        context_j = context_values[i + 1]
        only_in_i, in_both, only_in_j, shared_items = compare_context_artefacts(comparing_objects_df, context_i, context_j)

        results.append({
            'Objects ONLY in context i': [art for art, _ in only_in_i],
            'Objects in BOTH contexts': [art for art, _ in in_both],
            'Objects ONLY in context j': [art for art, _ in only_in_j]
        })

        shared_results.extend(shared_items)

    # DataFrames for both CSVs
    result_df = pd.DataFrame(results)
    shared_df = pd.DataFrame(shared_results, columns=['artefact', 'composite_id_i', 'composite_id_j'])

    return result_df, shared_df

# Assuming final_df is already defined in your Jupyter notebook...
comparing_objects_df = create_artefact_df(final_df)
results, shared_artifacts = iterate_context_pairs(comparing_objects_df)

# Export the first DataFrame results to CSV
results.to_csv('analyzed_folder/artefact_data/NEO4_comparing_contexts_artefacts.csv', index=False)

# Export the shared artifacts to another CSV
shared_artifacts.to_csv('analyzed_folder/artefact_data/NEO4_shared_artefacts_across_contexts.csv', index=False)

# Brainerd-Robinson Similarity Coefficient and Sampling Error Assessment


We use the code developed by [Matt Peeples](https://github.com/mpeeples2008/Brainerd-Robinson-Similarity-Coefficient-and-Sampling-Error-Assessment) with the R kernel here.

When the actual BR code runs, it will ask you if the data is in counts or percent; choose counts. Then use 1000 for the number of random runs in order to assess the probability of the similarity.

YOU MUST EXPAND THE CODE BLOCK and scroll down to see/interact with the user prompt.

In [29]:
import pandas as pd
import re

def clean_count_artefacts(csv_file):

    # 1. Read a csv file into a dataframe
    df = pd.read_csv(csv_file)

    # 2. Remove any digits from strings in the column 'Artefact'
    df['Artefact'] = df['Artefact'].apply(lambda x: re.sub(r'\d+', '', x))

    # 3. Group by values in the column 'context'
    group_data = df.groupby(['context', 'Artefact'])

    # 4. Show the count of each item in 'artefact' by 'context'. If there are duplicate items, sum their counts.
    group_data_counts = group_data.size().reset_index(name='Counts')

    # Transpose data so that context is the index, artefacts are the columns, fill N/A with 0
    result_data = group_data_counts.pivot(index='context', columns='Artefact', values='Counts').fillna(0)

    return result_data

def clean_subtypes_artefacts(csv_file):
    # 1. Read a csv file into a dataframe
    df = pd.read_csv(csv_file)

    # 2. Keep only the string after the second dash in the column 'Artefact'
    df['Artefact'] = df['Artefact'].apply(lambda x: x.split('-')[-1])

    # 2.1 Remove any digits from strings in the column 'Artefact'
    df['Artefact'] = df['Artefact'].apply(lambda x: re.sub(r'\d+', '', x))

    # 3. Group by values in the column 'context'
    group_data = df.groupby(['context', 'Artefact'])

    # 4. Show the count of each item in 'artefact' by 'context'. If there are duplicate items, sum their counts.
    group_data_counts = group_data.size().reset_index(name='Counts')

    # Transpose data so that context is the index, artefacts are the columns, fill N/A with 0
    result_data = group_data_counts.pivot(index='context', columns='Artefact', values='Counts').fillna(0)

    return result_data

In [30]:
BR_counts = clean_count_artefacts("analyzed_folder/artefact_data/processed_artefacts.csv")
BR_counts.to_csv('analyzed_folder/BR_counts.csv')

BR_subtype_counts = clean_subtypes_artefacts("analyzed_folder/artefact_data/processed_artefacts.csv")
BR_subtype_counts.to_csv("analyzed_folder/BR_subtype_counts.csv")

In [ ]:
%%R
# script by Matt Peeples, Matthew.Peeples@asu.edu, http://www.public.asu.edu/~mpeeple/

library(statnet) # initialize necessary library

# Function for calculating Brainerd-Robinson (BR) coefficients
BR <- function(x) {
rd <- dim(x)[1]
results <- matrix(0,rd,rd)
for (s1 in 1:rd) {
for (s2 in 1:rd) {
x1Temp <- as.numeric(x[s1, ])
x2Temp <- as.numeric(x[s2, ])
br.temp <- 0
results[s1,s2] <- 200 - (sum(abs(x1Temp - x2Temp)))}}
row.names(results) <- row.names(x)
colnames(results) <- row.names(x)
return(results)}

# Obtain input table
br.tab1 <- read.table(file='analyzed_folder/BR_counts.csv', sep=',', header=T, row.names=1) # the name of each row (site name) should be the first column in the input table
#br.tab1 <- read.table(file='analyzed_folder/BR_subtype_counts.csv', sep=',', header=T, row.names=1) # the name of each row (site name) should be the first column in the input table

#br.tab1 <- read.table(file='sq3_and_sq5_for_BR.csv', sep=',', header=T, row.names=1) # the name of each row (site name) should be the first column in the input table

br.tab1 <- na.omit(br.tab1)

# Ask for user if data are counts or percents
choose.per <- function(){readline("Are the type data percents or counts? 1=percents, 2=counts : ")}
per <- as.integer(choose.per())

# If user selects counts, convert data to percents and run simulation
if (per == 2) {
br.tab <- prop.table(as.matrix(br.tab1),1)*100
br.dat <- BR(br.tab) # actual BR values

# Calculate the proportions of each category in the original data table
c.sum <- matrix(0,1,ncol(br.tab1))
for (i in 1:ncol(br.tab1)) {
c.temp <- sum(br.tab1[,i])
c.sum[,i] <- c.temp}
p.grp <- prop.table(c.sum)

# Create random sample of a specified sample size
MC <- function(x,s.size) {
v3 <- matrix(0,ncol(x),1)
rand.tab.2 <- matrix(0,ncol(x),1)
v <- sample(ncol(x),s.size,prob=p.grp,replace=T)
v2 <- as.matrix(table(v))
for (j in 1:nrow(v2)) {
v3.temp <- v2[j,]
v3[j,1] <- v3.temp}
rand.tab <- as.matrix(prop.table(v3))*100
rand.tab.2[,1] <- rand.tab
return(rand.tab.2)}

r.sums <- as.matrix(rowSums(br.tab1)) # Calculate sample size by row

# Initate random samples for all rows
BR_rand <- function(x) {
rand.test <- matrix(0,ncol(x),nrow(r.sums))
for (i in 1:nrow(x)) {
rand.test[,i] <- MC(br.tab1,r.sums[i,])}
return(rand.test)}

br.diff.out <- matrix(0,nrow(br.dat),ncol(br.dat)) # Initialize table

# Ask for user how many random counts to calculate
choose.per <- function(){readline("How many random runs (1000 = recommended): ")}
randruns <- as.integer(choose.per())

# Run MC simulation of BR values
for (i in 1:randruns) {
br.temp <- BR_rand(br.tab1)
br.temp <- t(br.temp)
br.test <- BR(br.temp)
br.diff <- br.dat - br.test
br.diff2 <- event2dichot(br.diff,method='absolute',thresh=0)
br.diff.out <- br.diff.out + br.diff2}

br.diff.out <- br.diff.out / randruns # Calculate probabilities based on random runs
row.names(br.diff.out) <- row.names(br.tab1)
colnames(br.diff.out) <- row.names(br.tab1)

write.table(br.diff.out,file='analyzed_folder/BR_prob.csv',sep=',') # Write output

} # close if statement for count data

# Recalculate actual BR values and output to file
br.tab <- prop.table(as.matrix(br.tab1),1)*100
br.dat <- BR(br.tab)
write.table(br.dat,file='analyzed_folder/BR_out.csv',sep=',') # Write output

# end of script

In [ ]:
%%R
# script by Matt Peeples, Matthew.Peeples@asu.edu, http://www.public.asu.edu/~mpeeple/

library(statnet) # initialize necessary library

# Function for calculating Brainerd-Robinson (BR) coefficients
BR <- function(x) {
rd <- dim(x)[1]
results <- matrix(0,rd,rd)
for (s1 in 1:rd) {
for (s2 in 1:rd) {
x1Temp <- as.numeric(x[s1, ])
x2Temp <- as.numeric(x[s2, ])
br.temp <- 0
results[s1,s2] <- 200 - (sum(abs(x1Temp - x2Temp)))}}
row.names(results) <- row.names(x)
colnames(results) <- row.names(x)
return(results)}

# Obtain input table
#br.tab1 <- read.table(file='analyzed_folder/BR_counts.csv', sep=',', header=T, row.names=1) # the name of each row (site name) should be the first column in the input table
br.tab1 <- read.table(file='analyzed_folder/BR_subtype_counts.csv', sep=',', header=T, row.names=1) # the name of each row (site name) should be the first column in the input table

#br.tab1 <- read.table(file='sq3_and_sq5_for_BR.csv', sep=',', header=T, row.names=1) # the name of each row (site name) should be the first column in the input table

br.tab1 <- na.omit(br.tab1)

# Ask for user if data are counts or percents
choose.per <- function(){readline("Are the type data percents or counts? 1=percents, 2=counts : ")}
per <- as.integer(choose.per())

# If user selects counts, convert data to percents and run simulation
if (per == 2) {
br.tab <- prop.table(as.matrix(br.tab1),1)*100
br.dat <- BR(br.tab) # actual BR values

# Calculate the proportions of each category in the original data table
c.sum <- matrix(0,1,ncol(br.tab1))
for (i in 1:ncol(br.tab1)) {
c.temp <- sum(br.tab1[,i])
c.sum[,i] <- c.temp}
p.grp <- prop.table(c.sum)

# Create random sample of a specified sample size
MC <- function(x,s.size) {
v3 <- matrix(0,ncol(x),1)
rand.tab.2 <- matrix(0,ncol(x),1)
v <- sample(ncol(x),s.size,prob=p.grp,replace=T)
v2 <- as.matrix(table(v))
for (j in 1:nrow(v2)) {
v3.temp <- v2[j,]
v3[j,1] <- v3.temp}
rand.tab <- as.matrix(prop.table(v3))*100
rand.tab.2[,1] <- rand.tab
return(rand.tab.2)}

r.sums <- as.matrix(rowSums(br.tab1)) # Calculate sample size by row

# Initate random samples for all rows
BR_rand <- function(x) {
rand.test <- matrix(0,ncol(x),nrow(r.sums))
for (i in 1:nrow(x)) {
rand.test[,i] <- MC(br.tab1,r.sums[i,])}
return(rand.test)}

br.diff.out <- matrix(0,nrow(br.dat),ncol(br.dat)) # Initialize table

# Ask for user how many random counts to calculate
choose.per <- function(){readline("How many random runs (1000 = recommended): ")}
randruns <- as.integer(choose.per())

# Run MC simulation of BR values
for (i in 1:randruns) {
br.temp <- BR_rand(br.tab1)
br.temp <- t(br.temp)
br.test <- BR(br.temp)
br.diff <- br.dat - br.test
br.diff2 <- event2dichot(br.diff,method='absolute',thresh=0)
br.diff.out <- br.diff.out + br.diff2}

br.diff.out <- br.diff.out / randruns # Calculate probabilities based on random runs
row.names(br.diff.out) <- row.names(br.tab1)
colnames(br.diff.out) <- row.names(br.tab1)

write.table(br.diff.out,file='analyzed_folder/BR_subtypes_prob.csv',sep=',') # Write output

} # close if statement for count data

# Recalculate actual BR values and output to file
br.tab <- prop.table(as.matrix(br.tab1),1)*100
br.dat <- BR(br.tab)
write.table(br.dat,file='analyzed_folder/BR_subtype_out.csv',sep=',') # Write output

# end of script

# Initial Visualizations

We produce a heatmap for contexts against contexts and their BR similarity coefficients.

We produce voronoi diagram for artefacts by contexts.

In the 'voronoi' code block, you may wish to adjust line 13 for the list of major types.

Plots will be displayed below (click on '2 cells hidden'), and also written to the 'analyzed_folder'.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv("analyzed_folder/BR_out.csv")  # loading the data
sns.heatmap(df, cmap='coolwarm', linewidths=.5)
plt.suptitle('BR Coefficient of Similarity Matrix, by Type', fontsize=16)
plt.savefig('analyzed_folder/sq3_BR_by_type-300dpi.png', dpi=300,bbox_inches="tight")
plt.show()

df2 = pd.read_csv("analyzed_folder/BR_subtype_out.csv")  # loading the data
sns.heatmap(df2, cmap='coolwarm', linewidths=.5)
plt.suptitle('BR Coefficient of Similarity Matrix, by Subtype', fontsize=16)
plt.savefig('analyzed_folder/sq_3BR_by_subtype-300dpi.png', dpi=300,bbox_inches="tight")
plt.show()

In [ ]:
# voronoi by major subtypes, mosaic colouration
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial import Voronoi, voronoi_plot_2d
import seaborn as sns
import matplotlib.patches as mpatches
from shapely.geometry import Polygon
from shapely.ops import unary_union, polygonize
from scipy.spatial import Delaunay

#allowed_subtypes = ['restraint', 'tool', 'container', 'writing', 'experiment'] #sq3
allowed_subtypes = ['tool', 'container', 'writing', 'experiment'] #sq3 jan 10 with 'restraint' removed
#allowed_subtypes = ['restraint', 'body maintenance', 'computing', 'power', 'container'] #sq5
#allowed_subtypes = ['body maintenance', 'computing', 'power', 'container'] #sq5 Justin asks if 'restraint' removed, what does it look like

def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    df['subtype'] = df['Artefact'].str.split('-').str[2].str.replace('\d+', '', regex=True).str.lower()
    return df[df['subtype'].isin(allowed_subtypes)]

def plot(data: pd.DataFrame):
    data = preprocess_data(data)

    # Get unique subtypes and assign them a color
    subtype_colors = {subtype: color for subtype, color
                      in zip(data['subtype'].unique(), sns.color_palette('hsv', n_colors=data['subtype'].nunique()))}

    def voronoi_finite_polygons_2d(vor, radius=None):
        """
        Reconstruct infinite voronoi regions in a 2D diagram to finite
        regions.
        """
        if vor.points.shape[1] != 2:
            raise ValueError("Requires 2D input")

        new_regions = []
        new_vertices = vor.vertices.tolist()

        center = vor.points.mean(axis=0)
        if radius is None:
            radius = vor.points.ptp().max()

        # Construct a map containing all ridges for a given point
        all_ridges = {}
        for (p1, p2), (v1, v2) in zip(vor.ridge_points, vor.ridge_vertices):
            all_ridges.setdefault(p1, []).append((p2, v1, v2))
            all_ridges.setdefault(p2, []).append((p1, v1, v2))

        # Reconstruct infinite regions
        for p1, region in enumerate(vor.point_region):
            vertices = vor.regions[region]

            if all(v >= 0 for v in vertices):
                # Finite region
                new_regions.append(vertices)
                continue

            # Reconstruct a non-finite region
            ridges = all_ridges[p1]
            new_region = [v for v in vertices if v >= 0]

            for p2, v1, v2 in ridges:
                if v2 < 0:
                    v1, v2 = v2, v1
                if v1 >= 0:
                    # Finite ridge: already in the region
                    continue

                # Compute the missing endpoint of an infinite ridge
                t = vor.points[p2] - vor.points[p1]  # tangent
                t /= np.linalg.norm(t)
                n = np.array([-t[1], t[0]])  # normal

                midpoint = vor.points[[p1, p2]].mean(axis=0)
                direction = np.sign(np.dot(midpoint - center, n)) * n
                far_point = vor.vertices[v2] + direction * radius

                new_region.append(len(new_vertices))
                new_vertices.append(far_point.tolist())

            # Sort region counterclockwise.
            vs = np.asarray([new_vertices[v] for v in new_region])
            c = vs.mean(axis=0)
            angles = np.arctan2(vs[:, 1] - c[1], vs[:, 0] - c[0])
            new_region = np.array(new_region)[np.argsort(angles)]

            # Finish
            new_regions.append(new_region.tolist())

        return new_regions, np.asarray(new_vertices)

    fig, axs = plt.subplots(nrows=6, ncols=10, figsize=(20, 12))
    for i, ax in enumerate(axs.flatten()):
        if i < len(data['context'].unique()):
            context = data[data['context'] == i]
            points = context['centroid'].str.extract(r'\((.*),(.*)\)', expand=True).astype(float).values
            vor = Voronoi(points)
            regions, vertices = voronoi_finite_polygons_2d(vor)

            centroids = context[['centroid', 'subtype']]
            for region_index, region in enumerate(regions):
                polygon = vertices[region]
                # Fill the region with color
                color = subtype_colors[centroids.iloc[region_index]['subtype']]
                ax.fill(*zip(*polygon), color=color, alpha=0.4)

            voronoi_plot_2d(vor, ax=ax, show_vertices=False, line_colors='orange',
                            line_width=1, line_alpha=0.5, point_size=2)

            ax.set_title(f'Context {i}')
            ax.set_xlim(vor.min_bound[0] - 0.1, vor.max_bound[0] + 0.1)
            ax.set_ylim(vor.min_bound[1] - 0.1, vor.max_bound[1] + 0.1)
            ax.set_xticks([])
            ax.set_yticks([])
        else:
            ax.axis('off')

    # Create legend patches
    legend_patches = [mpatches.Patch(color=color, label=subtype) for subtype, color in subtype_colors.items()]
    # Add the legend to the plot
    plt.legend(handles=legend_patches, bbox_to_anchor=(1.05, 1), loc='upper left')

    plt.suptitle('Voronoi Map of Artefact Centroids by Context', fontsize=16)
    plt.tight_layout(rect=[0, 0, 0.85, 0.95])  # Make space for legend
    plt.savefig('analyzed_folder/voronoi-300dpi.png', dpi=300,bbox_inches="tight")
    return plt

data = pd.read_csv("analyzed_folder/artefact_data/processed_artefacts.csv")

chart = plot(data)
chart.show()

# Export
The next codeblock exports everything in the 'analyzed_folder'. It zips the folder up, and then downloads automatically. Be patient.

In [ ]:
!zip -r ISSAP-results.zip analyzed_folder/
from google.colab import files
files.download("ISSAP-results.zip")

# Clean UP

If you are going to analyze another json file in the current session, run the code below to clean up the workspace. YOU DO NOT NEED to run the initialization step, but you SHOULD change the input_file value to the name of your new json file, and then start by running from the Initial Data Extraction step downwards.

In [ ]:
! rm -r output_folder
! rm -r analyzed_folder
! rm -r closed_polygon_data